In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import geopandas as gpd
import scanpy as sc
import visualization_helper # Custom helper functions 
from PIL import Image # Converting image
import re # regex search
from pathlib import Path # handling different directories

from tifffile import imread, imwrite
from csbdeep.utils import normalize
from stardist.models import StarDist2D
from shapely.geometry import Polygon, Point
from scipy import sparse
from matplotlib.colors import ListedColormap

%matplotlib inline
%config InlineBackend.figure_format = 'retina'



2025-04-25 16:28:08.188191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745623688.202394   18332 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745623688.207103   18332 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745623688.225080   18332 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745623688.225103   18332 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745623688.225105   18332 computation_placer.cc:177] computation placer alr

# Todo
1. load the tissue parquet - tissue_position_file
2. Files to save
   - model output (labels, polynomials) - solo folder
        * labels as a .tif file - tifffile.imwrite(labels_output, labels.astype('uint16'))
        * polynomials - polys_output = labels_output.replace(".tif", "_polys.pkl") with open(polys_output, "wb") as f:    pickle.dump(polys, f)
   - Gdf file generated from the polynomials parater from the model, created with this: gdf = gpd.GeoDataFrame(geometry=geometries)  
   - parameters.txt - json_output
   - Save overlay figure
   - processed grouped_filtered_adata file
        * Figure out if this contains the polygon information and has the ID's 
3. Make sure to change the tiling to take the parameters defined below
4. Make sure that the IDing starts one ID value AFTER the previous, so I need to add a way to grab what the max ID value is and make sure that the next iteration starts with that. So this varaible needs to be updated in the loop and make it accessible in the next iteration - Here: gdf['id'] = [f"ID_{i+1}" for i, _ in enumerate(gdf.index)]
5. 

In [7]:
def ensure_dir(path):
    """Create the directory if it doesn't exist."""
    path = Path(path)
    if not path.exists():
        path.mkdir(parents=True, exist_ok=True)
        print(f"Created directory: {path}")
    else:
        print(f"Directory already exists: {path}")

In [26]:
# parameters

#Image normalization
low_percentile = 0.5 # default 1
upper_percentile = 99.9 # default 99.8

#Model Parameters
model_scale = 7#7       # Default values 1
nms_threshold = 0.1   # 0.3
prob_threshold = 0.3 
n_tiles = 20

# For this:
# img_norm = normalize(img_np, low_percentile, upper_percentile, axis=(0, 1, 2))

In [9]:
files = [
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07833_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07834_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07835_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07836_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07837_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07838_22WJCYLT3"
]

# file = files[1]

dir_base = Path(
    '/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/'
    'dietary_droject/data/Rose_Li_VisiumHD/'
)

segmentation_path = Path("/mnt/c/Users/jonan/Documents/1Work/"
                         "RoseLab/Spatial/dietary_droject/data/"
                         "cell_segmentation/SegmentedData")

# tissue_image_loc = dir_base / file / 'outs/spatial/tissue_hires_image.png'

for file in files:
    print(f'Full sample folder: {file}')

    # Grabbing f number
    f_number = re.search(r'_F\d+_', file).group(0).strip('_')
    print(f'Sample number: {f_number}')

    #hires tissue location
    tissue_image_loc = dir_base / file / 'outs/spatial/tissue_hires_image.png'
    # print(f'Location of tissue image: {tissue_image_loc}')
    if tissue_image_loc.exists():
        print("hires exists")
    else:
        print("hires exists not exist")


    h5_file_2um = dir_base / file / "outs/binned_outputs/square_002um/filtered_feature_bc_matrix.h5"
    if h5_file_2um.exists():
        print("h5 file exists")
    else:
        print("h5 does not exist")

    parquet_file_2um = dir_base / file / 'outs/binned_outputs/square_002um/spatial/tissue_positions.parquet'
    if parquet_file_2um.exists():
        print("parquet file exists \n")
    else: 
        print("parquet file does not exists \n")


    #====================================== Out put Stuff ================================================

    
    output_root = segmentation_path / f_number
    mdl_output = output_root / "model_output"
    fig_output = output_root / "figures"

    ensure_dir(output_root)
    ensure_dir(mdl_output)
    ensure_dir(fig_output)




    
    

Full sample folder: BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07833_22WJCYLT3
Sample number: F07833
hires exists
h5 file exists
parquet file exists 

Created directory: /mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/dietary_droject/data/cell_segmentation/SegmentedData/F07833
Created directory: /mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/dietary_droject/data/cell_segmentation/SegmentedData/F07833/model_output
Created directory: /mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/dietary_droject/data/cell_segmentation/SegmentedData/F07833/figures
Full sample folder: BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07834_22WJCYLT3
Sample number: F07834
hires exists
h5 file exists
parquet file exists 

Created directory: /mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/dietary_droject/data/cell_segmentation/SegmentedData/F07834
Created directory: /mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/dietary_droject/data/cell_segmentation/SegmentedData/F07834/model_output
Created directory: /mnt/c/

In [ ]:
# Testing output folders

# Compile a list of .txt files that 


In [ ]:

files = [
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07833_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07834_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07835_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07836_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07837_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07838_22WJCYLT3"
]

dir_base = (
    '/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/'
    'dietary_droject/data/Rose_Li_VisiumHD/'
    'BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07833_22WJCYLT3/outs/spatial/'
)


filename = 'tissue_hires_image.png'

img_pil = Image.open(dir_base + filename).convert("RGB")
img_np = np.array(img_pil)

# Normalize across all channels
img_norm = normalize(img_np, low_percentile, upper_percentile, axis=(0, 1, 2))
print(img_np.shape)